### Installs

In [ ]:
!pip install segmentation_models_pytorch
!git clone https://github.com/Bjarten/early-stopping-pytorch.git
!mv ./early-stopping-pytorch ./lib

### Imports

In [ ]:
import torch, torchvision
import matplotlib.pyplot as plt
import time
import copy
import random
import os 
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models


import math
import sys
import cv2

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.pyplot as mpplot
from sklearn.model_selection import train_test_split
from PIL import Image

import segmentation_models_pytorch as smp
from lib.pytorchtools import *

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.backends.cudnn.deterministic = True

In [ ]:
path = "/kaggle/input/ultrasound-nerve-segmentation/"

train_path = os.path.join(path, "train")
test_path = os.path.join(path, "test")

masks = [os.path.join(train_path,i) for i in os.listdir(train_path) if "mask" in i]
imgs = [i.replace("_mask","") for i in masks]

df = pd.DataFrame({"images":imgs,"masks":masks})

train_df, val_df = train_test_split(df,test_size = 0.20)

train_df.head()

### Dataset

In [ ]:
class Dataset(Dataset):
    def __init__(self, df, transform=None):
        self.images = df.images.tolist()
        self.masks = df.masks.tolist()
        self.transform = transform
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        image_name = self.images[idx]
        mask_name = self.masks[idx]
       
        image = Image.open(image_name)
        mask = Image.open(mask_name)  
    
        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)
    
        return image, mask

In [ ]:
data_transforms = transforms.Compose([
    transforms.Resize(size=(224, 224)),
    transforms.ToTensor()
])

batch_size = 16

train_dataset = Dataset(train_df, data_transforms)
val_dataset = Dataset(val_df, data_transforms)

train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, num_workers=batch_size)

val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, num_workers=batch_size)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print('train', len(train_dataloader), len(train_dataset))
print('val', len(val_dataloader), len(val_dataset))

In [ ]:
def show_input(data, n):
    fig = plt.figure(figsize=(5, 5))
        
    for i in range(1, n + 1):
        img_ax = fig.add_subplot(2, n, i)
        msk_ax = fig.add_subplot(2, n, i + n)
        
        image = data[i-1][0].permute(1, 2, 0).numpy()
        mask = data[i-1][1].permute(1, 2, 0).numpy()
        
        img_ax.imshow(image, cmap='gray')
        msk_ax.imshow(mask, cmap='gray')
        
    fig.show()

show_input(train_dataset, 5)

### Model

In [ ]:
model = smp.Unet(
    encoder_name='resnet34',
    encoder_weights='imagenet',
    in_channels=1,
    classes=1,
    activation='sigmoid'
)

loss = smp.utils.losses.DiceLoss()
metrics = [smp.utils.metrics.IoU()]

learning_rate = 0.001
epochs = 50

stopper = EarlyStopping(patience=3)


optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)

loss_function = smp.utils.losses.DiceLoss()

train_epoch = smp.utils.train.TrainEpoch(model,
                                          loss=loss_function,
                                          optimizer=optimizer,
                                          metrics=metrics,
                                          device=device,
                                          verbose=True)
val_epoch = smp.utils.train.ValidEpoch(model,
                                          loss=loss_function,
                                          metrics=metrics,
                                          device=device,
                                          verbose=True)

### Train

In [ ]:
model.to(device)
train_loss_history = []
val_loss_history= []

train_iou_history = []
val_iou_history = []

for epoch in range(epochs):
    print('\nEpoch: {}'.format(epoch))
    train_log = train_epoch.run(train_dataloader)
    val_log = val_epoch.run(val_dataloader)

    scheduler.step()

    train_loss_history.append(train_log[loss_function.__name__])
    val_loss_history.append(val_log[loss_function.__name__])

    train_iou_history.append(train_log['iou_score']) 
    val_iou_history.append(val_log['iou_score'])

    stopper(val_log[loss_function.__name__], model)
    if stopper.early_stop:
        break

### Vizualize

In [ ]:
def visualize_train(train, val, title):
    plt.plot(range(len(train)), train, label = 'Train')
    plt.plot(range(len(val)), val, label = 'Val')
    
    plt.ylabel(title)
    plt.xlabel('epoch')
    
    plt.legend(title)
    plt.show()

In [ ]:
visualize_train(train_loss_history, val_loss_history, 'Loss')

In [ ]:
visualize_train(train_iou_history, val_iou_history, 'IoU')

In [ ]:
def rle_encoding(x):
    dots = np.where(x.T.flatten()==1)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b+1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths

In [ ]:
imgs = [f for f in os.listdir(test_path)]
imgs = sorted( imgs, key=lambda s: int(s.split('.')[0]))

submission_df = pd.DataFrame(columns=['img', 'pixels'])
model.to(device)
model.eval()

for i, img in enumerate(tqdm(imgs)):
    x = Image.open(os.path.join(test_path, img))

    x = data_transforms(x)

    x = x.unsqueeze(0).to(device)
    prediction = model.predict(x)

    prediction = prediction.cpu()
    prediction = transforms.Resize(size=(420, 580))(prediction)

    encoding = rle_encoding(prediction)

    pixels = ' '.join(map(str, encoding))
    submission_df.loc[i] = [str(i+1), pixels]

submission_df.to_csv('submission.csv', index=False)